In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
import sys
sys.path.append("/content/drive/MyDrive/fairness_discrepancy")

In [26]:
import utils as ut
import loss_funcs as lf
import numpy as np
from prep_adult_data import *
from contextlib import redirect_stdout

In [27]:
""" Load the adult data """
X, y, x_control = load_adult_data(load_data_size=10000) # set the argument to none, or no arguments if you want to test with the whole data -- we are subsampling for performance speedup
ut.compute_p_rule(x_control["sex"], y) # compute the p-rule in the original data



""" Split the data into train and test """
train_fold_size = 0.7
x_train, y_train, x_control_train, x_test, y_test, x_control_test = ut.split_into_train_test(X, y, x_control, train_fold_size)

loss_function = lf.discrepancy_loss
sensitive_attrs = ["sex"]

Looking for file 'adult.data' in the current directory...
File found in current directory..
Looking for file 'adult.test' in the current directory...
File found in current directory..
Loading only 10000 examples from the data
Total data points: 10000
# non-protected examples: 6786
# protected examples: 3214
Non-protected in positive class: 2110 (31%)
Protected in positive class: 364 (11%)
P-rule is: 36%


In [28]:
def train_test_classifier():
    
#     final_c, Cs, losses = ut.train_model(x_train, y_train, x_control_train, loss_function,sensitive_attrs, max_iter, alpha, l, method, initiator, catol)
    final_c, Cs, losses = ut.train_model(x_train, y_train, x_control_train, loss_function, sensitive_attrs, max_iter, 
                                            alpha, l, method, initiator, catol)
    y_train_predicted, y_test_predicted = ut.predict(final_c.x, x_train, y_train, x_test)
    train_score, test_score, correct_answers_train, correct_answers_test = \
                                            ut.check_accuracy(None, x_train, y_train, 
                                                              x_test, y_test, y_train_predicted, y_test_predicted)
    print("Train data:")
    print("------------")
    print("Train accuracy : ", train_score)
    p_rule_train = ut.compute_p_rule(x_control_train["sex"], y_train_predicted)
    print()
    print("Test data: ")
    print("------------")
    print("Test accuracy : ", test_score)
    p_rule_test = ut.compute_p_rule(x_control_test["sex"], y_test_predicted)
    print("------------------------------------------------------------------------")
    print("------------------------------------------------------------------------")
    print
    print
    return Cs, losses, train_score, test_score, p_rule_train, p_rule_test

In [29]:
method = 'cobyla'
# l_list = np.linspace(start= 0.00000001, stop= 10, num= 5)     # lamda list
alpha_list = np.linspace(start=0.0 , stop= 1.0, num= 10)
# initiator_list = np.linspace(start= 0, stop= 0.01, num= 10) # for initial values of c
max_iter_list = np.linspace(start= 100, stop= 1000, num= 10) # no of iterations
# catol_list = np.linspace(start=0, stop= 0.01, num= 5)
initiator = 0.001
l = 1
catol = 0.0001

In [ ]:
i = 0
result = {'l':[], 'alpha':[], 'initiator':[], 'max_iter':[], 'catol':[], 'c':[], 'loss':[],
                         'train_accuracy':[], 'test_accuracy':[], 'p_rule_train':[],
                             'p_rule_test':[]}

In [ ]:
for alpha in alpha_list:
    for max_iter in max_iter_list:
        print(i)
        with open('out_colab'+ str(i) +'.log', 'w') as f:
            with redirect_stdout(f):
                c, loss, train_accuracy, test_accuracy, p_rule_train,\
                                                p_rule_test = train_test_classifier()
                result['l'].append(l)
                result['alpha'].append(alpha)
                result['max_iter'].append(max_iter)
                result['initiator'].append(initiator)
                result['catol'].append(catol)
                result['c'].append(c)
                result['loss'].append(loss)
                result['train_accuracy'].append(train_accuracy)
                result['test_accuracy'].append(test_accuracy)
                result['p_rule_train'].append(p_rule_train)
                result['p_rule_test'].append(p_rule_test)
        i = i+1

0


KeyboardInterrupt: ignored

In [6]:
%load_ext autoreload
%autoreload 2
alpha = .5
max_iter = 10
# initiator=10
%time train_test_classifier()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
iter:  10   lambda:  1   alpha:  0.5   kernel: rbf, method:  cobyla
x_init:  [4.61210947e-05 1.54633456e-04 2.19932822e-04 ... 5.61520195e-05
 7.17752571e-04 3.05347832e-04]
loss:  910.7612513365544
loss:  910.5146603156919
loss:  910.264981719186
loss:  910.0191902559765
loss:  909.7697142952132
loss:  909.5204259625265
loss:  909.2711384222831
loss:  909.0212401444286
loss:  908.7707912656919
loss:  908.521165183762
weights:  [1.00004612e+00 1.00015463e+00 1.00021993e+00 ... 5.61520195e-05
 7.17752571e-04 3.05347832e-04]
Optimization problem did not converge.. Check the solution returned by the optimizer.
Returned solution is:
     fun: 908.521165183762
   maxcv: 6.805539396043335
 message: 'Did not converge to a solution satisfying the constraints. See `maxcv` for magnitude of violation.'
    nfev: 10
  status: 4
 success: False
       x: array([1.00004612e+00, 1.00015463e+00, 1.00021993e+00, ...

([array([6.95212107e-310, 6.95212107e-310, 1.56324030e-316, ...,
         5.61520195e-005, 7.17752571e-004, 3.05347832e-004]),
  array([6.95212107e-310, 6.95212107e-310, 1.56324030e-316, ...,
         5.61520195e-005, 7.17752571e-004, 3.05347832e-004]),
  array([6.95212107e-310, 6.95212107e-310, 1.56324030e-316, ...,
         5.61520195e-005, 7.17752571e-004, 3.05347832e-004]),
  array([6.95212107e-310, 6.95212107e-310, 1.56324030e-316, ...,
         5.61520195e-005, 7.17752571e-004, 3.05347832e-004]),
  array([6.95212107e-310, 6.95212107e-310, 1.56324030e-316, ...,
         5.61520195e-005, 7.17752571e-004, 3.05347832e-004]),
  array([6.95212107e-310, 6.95212107e-310, 1.56324030e-316, ...,
         5.61520195e-005, 7.17752571e-004, 3.05347832e-004]),
  array([6.95212107e-310, 6.95212107e-310, 1.56324030e-316, ...,
         5.61520195e-005, 7.17752571e-004, 3.05347832e-004]),
  array([6.95212107e-310, 6.95212107e-310, 1.56324030e-316, ...,
         5.61520195e-005, 7.17752571e-004, 3.0